In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


# function for calculating distance
# This function is acquired online from:
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
def distance(lat1, lon1, lat2, lon2):
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    R = 6373
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (np.sin(dlat/2))**2 + np.cos(lat1) * np.cos(lat2) * (np.sin(dlon/2))**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c


# add a new column for distance calculated
df['distance'] = distance(df['start station latitude'], df['start station longitude'], 
                          df['end station latitude'], df['end station longitude'])

# add a new column age
df['age'] = 2018 - df['birth year']

# drop if distance = 0 but tripduration > 0
df.drop(df[(df['distance'] == 0) & (df['tripduration'] > 0)].index, inplace = True)


# histogram of distance without grouping
df.distance.hist(bins=80, figsize=(15,5))
plt.xlabel('Distance')
plt.title('Histogram of trip distances in miles')
df.distance.describe()

<li>We can see from above histogram that the bike trips are primarily short-distance trips. There are few trips longer than 3 miles.

In [ ]:
# histogram of distance and time with grouping of age, gender, and usertype
df.groupby('gender')['distance', 'tripduration'].mean()
df.groupby('age')['distance', 'tripduration'].mean()
df.groupby('usertype')['distance', 'tripduration'].mean()

<li> Bar charts on NYC Citi Bike trips by hour on weekdays and weekends separately

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import datetime

#formatting data
df['year'] = df.starttime.apply(lambda t: t.year())
df['month'] = df.starttime.apply(lambda t: t.month())
df['day'] = df.starttime.apply(lambda t: t.weekday())
df['hour'] = df.starttime.apply(lambda t: t.hour())

#pick up density by months
group_month = df.groupby('month')
group_month.size().plot(kind='bar')

#pick up density by hour on weekdays
df_weekday=df[df[day==(0|1|2|3|4)]]
group_weekday = df_weekday.groupby('hour')
group_weekday.size().plot(kind='bar')

#pick up density by hour on weekends
df_weekend=df[df[day==(5|6)]]
group_weekend = df_weekend.groupby('hour')
group_weekend.size().plot(kind='bar')